![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Working Memory with Long-Term Extraction Strategies

## Introduction

This notebook demonstrates how to implement **working memory** with configurable **long-term extraction strategies** that inform memory management tools about when and how to extract important information from working memory to long-term storage.

### Key Concepts

- **Working Memory**: Persistent storage for task-focused context (conversation messages, task-related data)
- **Long-term Memory**: Cross-session knowledge (user preferences, important facts learned over time)
- **Long-Term Extraction Strategy**: Configurable logic for when/how to move important information from working to long-term memory
- **Strategy-Aware Tools**: Memory tools that understand the extraction strategy and make intelligent decisions
- **Context-Informed LLM**: The LLM receives information about the extraction strategy to make better memory management decisions

### The Problem We're Solving

Previously, memory tools like `add_memories_to_working_memory` and `create_memory` operated without knowledge of:
- When memories should be extracted from working memory
- What criteria determine memory importance
- How the working memory's extraction strategy affects tool behavior

This notebook shows how to solve this by making tools **extraction strategy aware**.

In [ ]:
# Install the Redis Context Course package
import subprocess
import sys
import os

# Install the package in development mode
package_path = "../../reference-agent"
result = subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-e", package_path], 
                      capture_output=True, text=True)
if result.returncode == 0:
    print("✅ Package installed successfully")
else:
    print(f"❌ Package installation failed: {result.stderr}")
    raise RuntimeError(f"Failed to install package: {result.stderr}")

In [ ]:
import os
import sys

# Set up environment - handle both interactive and CI environments
def _set_env(key: str):
    if key not in os.environ:
        # Check if we're in an interactive environment
        if hasattr(sys.stdin, 'isatty') and sys.stdin.isatty():
            import getpass
            os.environ[key] = getpass.getpass(f"{key}: ")
        else:
            # Non-interactive environment (like CI)
            print(f"⚠️  {key} not found in environment. Some features may not work.")
            pass  # Let it fail if key is actually needed

_set_env("OPENAI_API_KEY")

# Set Redis URL
os.environ["REDIS_URL"] = "redis://localhost:6379"

## Working Memory Components

Let's explore the key components of our working memory system:

In [ ]:
# Import memory components
from redis_context_course import MemoryClient
from langchain_core.messages import HumanMessage, AIMessage

print("✅ Memory components imported successfully")
print("\nNote: This notebook demonstrates working memory concepts.")
print("The MemoryClient provides working memory via put_working_memory() and get_or_create_working_memory()")

## 1. Long-Term Extraction Strategies

Extraction strategies define **when** and **how** memories should be moved from working memory to long-term storage:

**Conceptual Example: Extraction Strategies**

In a production system, you would define extraction strategies that determine when to move memories from working to long-term storage:

```python
# Strategy 1: Message Count Strategy
strategy1 = MessageCountStrategy(message_threshold=5, min_importance=0.6)
# Triggers extraction after 5 messages, only for memories with importance >= 0.6

# Strategy 2: More aggressive extraction
strategy2 = MessageCountStrategy(message_threshold=3, min_importance=0.4)
# Triggers extraction after 3 messages, with lower importance threshold
```

**Importance Calculation Examples:**
- "I prefer online courses" → importance: 0.85 (preference)
- "My goal is to become a data scientist" → importance: 0.90 (goal)
- "What time is it?" → importance: 0.10 (trivial)
- "I love machine learning and want to specialize in it" → importance: 0.95 (strong preference + goal)
- "The weather is nice today" → importance: 0.15 (small talk)

The Agent Memory Server automatically handles this extraction when you save working memory.

## 2. Working Memory in Action

Let's see how working memory operates with an extraction strategy:

In [ ]:
# Initialize memory client for working memory
student_id = "demo_student_working_memory"
session_id = "session_001"

# The MemoryClient handles working memory automatically
# Initialize memory client with proper config
import os
config = MemoryClientConfig(
    base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8000"),
    default_namespace="redis_university"
)
memory_client = MemoryClient(config=config)

print("✅ Memory client initialized successfully")
print(f"📊 User ID: {student_id}")
print(f"📊 Session ID: {session_id}")
print("\nThe Agent Memory Server automatically extracts important information")
print("from working memory to long-term storage.")

In [ ]:
# Simulate a conversation using working memory
from agent_memory_client import MemoryAPIClient as MemoryClient, MemoryClientConfig

# Ensure memory_client and session_id are defined (in case cells are run out of order)
if 'memory_client' not in globals():
    # Initialize memory client with proper config
    import os
    config = MemoryClientConfig(
        base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8000"),
        default_namespace="redis_university"
    )
    memory_client = MemoryClient(config=config)
if 'session_id' not in globals():
    session_id = "session_001"

print("💬 Simulating Conversation with Working Memory")
print("=" * 50)

# Create messages for the conversation
messages = [
    {"role": "user", "content": "I prefer online courses because I work part-time"},
    {"role": "assistant", "content": "I understand you prefer online courses due to your work schedule."},
    {"role": "user", "content": "My goal is to specialize in machine learning"},
    {"role": "assistant", "content": "Machine learning is an excellent specialization!"},
    {"role": "user", "content": "What courses do you recommend?"},
]

# Save to working memory
from agent_memory_client import WorkingMemory, MemoryMessage

# Convert messages to MemoryMessage format
memory_messages = [MemoryMessage(**msg) for msg in messages]

# Create WorkingMemory object
working_memory = WorkingMemory(
    session_id=session_id,
    user_id="demo_user",
    messages=memory_messages,
    memories=[],
    data={}
)

await memory_client.put_working_memory(
    session_id=session_id,
    memory=working_memory,
    user_id="demo_user",
    model_name="gpt-4o"
)

print("✅ Conversation saved to working memory")
print(f"📊 Messages: {len(messages)}")
print("\nThe Agent Memory Server will automatically extract important information")
print("like preferences and goals to long-term memory.")

# Retrieve working memory
_, working_memory = await memory_client.get_or_create_working_memory(
    session_id=session_id,
    model_name="gpt-4o"
)

if working_memory:
    print(f"\n📋 Retrieved {len(working_memory.messages)} messages from working memory")

## 3. Memory Tools with Agent Memory Server

The Agent Memory Server provides tools for managing memories. You can use the built-in tools from the `redis_context_course` package:

In [ ]:
# Import memory tools
from redis_context_course import create_memory_tools, MemoryClient

# Ensure memory_client is defined (in case cells are run out of order)
if 'memory_client' not in globals():
    # Initialize memory client with proper config
    import os
    config = MemoryClientConfig(
        base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8000"),
        default_namespace="redis_university"
    )
    memory_client = MemoryClient(config=config)

# Create memory tools for this user
memory_tools = create_memory_tools(memory_client)

print("🛠️  Available Memory Tools")
print("=" * 50)

for tool in memory_tools:
    print(f"📋 {tool.name}")
    print(f"   Description: {tool.description.split('.')[0]}...")
    print()

print("\nThese tools allow the LLM to:")
print("- Store important information explicitly")
print("- Search for relevant memories")
print("- Control what gets remembered")
print("\nSee notebook 04_memory_tools.ipynb for detailed examples.")

## 4. Automatic Extraction by Agent Memory Server

The Agent Memory Server automatically extracts important information from working memory to long-term storage. You don't need to manually configure extraction strategies - it's handled automatically based on the content and context of the conversation.

In [ ]:
# Check what was extracted to long-term memory
import asyncio
from agent_memory_client import MemoryAPIClient as MemoryClient, MemoryClientConfig

# Ensure memory_client is defined (in case cells are run out of order)
if 'memory_client' not in globals():
    # Initialize memory client with proper config
    import os
    config = MemoryClientConfig(
        base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8000"),
        default_namespace="redis_university"
    )
    memory_client = MemoryClient(config=config)

await asyncio.sleep(2)  # Give the extraction process time to complete

# Search for extracted memories
extracted_memories = await memory_client.search_long_term_memory(
    query="preferences goals",
    limit=10
)

print("🧠 Extracted to Long-term Memory")
print("=" * 50)

if extracted_memories.memories:
    for i, memory in enumerate(extracted_memories.memories, 1):
        print(f"{i}. {memory.text}")
        print(f"   Type: {memory.memory_type} | Topics: {', '.join(memory.topics)}")
        print()
else:
    print("No memories extracted yet (extraction may take a moment)")
    print("\nThe Agent Memory Server extracts:")
    print("- User preferences (e.g., 'prefers online courses')")
    print("- Goals (e.g., 'wants to specialize in machine learning')")
    print("- Important facts (e.g., 'works part-time')")

## 5. Summary

In this notebook, you learned:

- ✅ Working memory stores session-scoped conversation context
- ✅ The Agent Memory Server automatically extracts important information
- ✅ Extraction happens asynchronously in the background
- ✅ You can provide memory tools to give the LLM explicit control
- ✅ The MemoryClient provides a simple API for working memory operations

**Key API Methods:**
```python
# Save working memory
await memory_client.save_working_memory(session_id, messages)

# Retrieve working memory
working_memory = await memory_client.get_working_memory(session_id, model_name)

# Search long-term memories
memories = await memory_client.search_memories(query, limit)
```

See the next notebooks for more on long-term memory and memory integration!

## Key Benefits

### ✅ **Strategy Awareness**
- Memory tools understand the current extraction strategy
- Tools can make intelligent decisions about memory placement
- LLM receives context about when extraction will happen

### ✅ **Intelligent Memory Management**
- High-importance memories can bypass working memory
- Extraction happens automatically based on configurable triggers
- Memory tools coordinate with extraction strategy

### ✅ **Configurable Behavior**
- Different extraction strategies for different use cases
- Importance calculation can be customized
- Trigger conditions are flexible and extensible

### ✅ **Context-Informed Decisions**
- Tools include strategy context in their descriptions
- LLM can make better decisions about memory management
- System prompt includes working memory status

## Next Steps

This working memory system with extraction strategy awareness provides a foundation for:

1. **Custom Extraction Strategies**: Implement time-based, importance-threshold, or conversation-end strategies
2. **Advanced Importance Calculation**: Use NLP techniques for better importance scoring
3. **Multi-Modal Memory**: Extend to handle different types of content (text, images, etc.)
4. **Memory Hierarchies**: Implement multiple levels of memory with different retention policies

The key insight is that **memory tools should be aware of the memory management strategy** to make intelligent decisions about when and how to store information.